In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("https://github.com/christianw/applied-ux/raw/main/week03/amazon_reviews.csv.xz", low_memory=False)

In [ ]:
pos_mask = df["name"] == "Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, Magenta"

In [ ]:
pos = df[pos_mask].copy()
pos["ft7"] = 1

In [ ]:
len(pos)

In [ ]:
neg = df[~pos_mask].copy()
neg["ft7"] = 0

In [ ]:
len(neg)

In [ ]:
labeled = pd.concat([pos, neg.sample(len(pos), random_state=42)])

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS as stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1,2), stop_words=list(stop_words), 
                        max_df=0.7, min_df=3)
tfidf_vectors = tfidf.fit_transform(labeled["reviews.text"].map(str))

X = tfidf_vectors
Y = labeled["ft7"]

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = \
        train_test_split(X, Y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

clf = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)
#clf = SVC(kernel='linear', random_state=42)
#clf = SVC(random_state=42)
clf.fit(X_train, Y_train)

In [ ]:
Y_predicted = clf.predict(X_test)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(Y_test, Y_predicted))